# 任务6 比赛整理(2天)

以赛方最后给的答辩模板为主线整理比赛思路，模拟比赛答辩环节，进行比赛整理。

## Part1

**参赛队成员简介**

 （这边主要介绍成员情况，如果有竞赛获奖就最好啦）

ps：这个由于是模拟比赛所以这个部分可以不写哦

## Part2

**参赛作品概述**

## Part3

**关键技术阐述（数据清洗、特征工程、模型、模型融合，并强调对比赛提分最有帮助的部分）**

## Part4

**探索与创新（写明做的与众不同的创新点）**

## Part5

**实施与优化过程（在过程中尝试过的方法都可以提及并总结）**

## Part6

**其他（有其他需要补充的可以写在这个部分）**



**（注：因为比赛是和企业合作，并具有实际意义的比赛，所以强调你的代码模型的实际意义，商业价值都会在答辩环节有帮助哦）**





**任务时间 2天**


**参考连接：**
[https://blog.csdn.net/qq_39756719/article/details/95634744](https://blog.csdn.net/qq_39756719/article/details/95634744)

# Part2
## 参赛作品概述
从赛题分析、数据清洗、特征工程、模型选择、模型融合几个方面介绍。

# Part3
# 关键技术阐述
## 1. 赛题分析
分析数据，查看数据整体情况。找出离散型特征和连续型特征；分析缺失值；分析单调特征、特征unique分布；

查看房屋面积分布；

![1](https://github.com/universecool/DataWhale/blob/master/pic/task6/1.png)

查看label分布；

![2](https://github.com/universecool/DataWhale/blob/master/pic/task6/2.png)

## 2. 数据清洗
数据清洗主要包括缺失值分析及处理、异常值分析及处理、深度清洗。

### **缺失值处理**

- **缺失值处理**

缺失值的处理手段大体可以分为：删除、填充、映射到高维(当做类别处理)。  
详细的请自行查找相关资料学习。  
根据任务一，直接找到的缺失值情况是pu和pv；但是，根据特征nunique分布的分析，可以发现rentType存在"--"的情况，这也算是一种缺失值。  
此外，诸如rentType的"未知方式"；houseToward的"暂无数据"等，本质上也算是一种缺失值，但是对于这些缺失方式，我们可以把它当做是特殊的一类处理，而不需要去主动修改或填充值。  
  
  将rentType的"--"转换成"未知方式"类别；  
  pv/pu的缺失值用均值填充；  
  buildYear存在"暂无信息"，将其用众数填充。  
  
- **转换object类型数据**  
这里直接采用LabelEncoder的方式编码，详细的编码方式请自行查阅相关资料学习。  
  
  
- **时间字段的处理**  
buildYear由于存在"暂无信息",所以需要主动将其转换int类型；  
tradeTime，将其分割成月和日。  
  
  
- **删除无关字段**  
ID是唯一码，建模无用，所以直接删除；  
city只有一个SH值，也直接删除；  
tradeTime已经分割成月和日，删除原来字段

###**异常值处理**

主要针对area和tradeMoney两个维度处理。  
针对tradeMoney，采用IsolationForest模型自动处理；  
针对areahetotalFloor采用主观+数据可视化的方式。

![3](https://github.com/universecool/DataWhale/blob/master/pic/task6/3.png)

### **深度清洗**

针对每一个region的数据，对area和tradeMoney两个维度进行深度清洗。 
采用主观+数据可视化的方式。

![4](https://github.com/universecool/DataWhale/blob/master/pic/task6/4.png)

## 3. 特征工程
对一些特征进行合并，如小区内所有交通站占总数的比例、健身房占总数的比例等、医院占总数的比例等；计算每平方米的价格；按社区划分的学校数量、医院的数量等；对相似特征进行聚类；对过大量级去log平滑。

然后选择有意义的特征。通常来说，从两个方面考虑来选择特征：

- 特征是否发散：如果一个特征不发散，例如方差接近于0，也就是说样本在这个特征上基本上没有差异，这个特征对于样本的区分并没有什么用。
- 特征与目标的相关性：这点比较显见，与目标相关性高的特征，应当优选选择。

根据特征选择的形式又可以将特征选择方法分为3种：
- Filter：过滤法，按照发散性或者相关性对各个特征进行评分，设定阈值或者待选择阈值的个数，选择特征。
- Wrapper：包装法，根据目标函数（通常是预测效果评分），每次选择若干特征，或者排除若干特征。
- Embedded：嵌入法，先使用某些机器学习的算法和模型进行训练，得到各个特征的权值系数，根据系数从大到小选择特征。类似于Filter方法，但是是通过训练来确定特征的优劣。

特征选择主要有两个目的：

- 减少特征数量、降维，使模型泛化能力更强，减少过拟合；
- 增强对特征和特征值之间的理解。

拿到数据集，一个特征选择方法，往往很难同时完成这两个目的。通常情况下，选择一种自己最熟悉或者最方便的特征选择方法（往往目的是降维，而忽略了对特征和数据理解的目的）。[1]

[1] https://www.cnblogs.com/stevenlk/p/6543628.html



# Part4
# 模型选择
单模型选用lightgbm， lightgbm是一种梯度提升决策树模型，采用boosting集合方式。LightGBM在很多方面会比XGBoost表现的更为优秀。它有以下优势：[2]

更快的训练效率
低内存使用
更高的准确率
支持并行化学习
可处理大规模数据
支持直接使用category特征

[2] https://www.biaodianfu.com/lightgbm.html

# Part 5
# 模型融合

将特征放进模型中预测，并将预测结果作为新的特征加入原有特征中再经过模型预测结果（可以反复预测多次将结果加入最后的特征中）。

采用Stacking方式融合模型结果。